In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb  8 12:51:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!yes y | pip uninstall torchy torchvision


Found existing installation: torch 1.10.0+cu111
Uninstalling torch-1.10.0+cu111:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.10.0+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.10.0+cu111
Found existing installation: torchvision 0.11.1+cu111
Uninstalling torchvision-0.11.1+cu111:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchvision-0.11.1+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libcudart.7be20469.so.11.0
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libnvjpeg.8313e8da.so.11
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/lib/pyth

In [3]:
!yes y | pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:27tcmalloc: large alloc 1147494400 bytes == 0x55a058c50000 @  0x7f94aff8d615 0x55a01f3a53bc 0x55a01f48618a 0x55a01f3a81cd 0x55a01f49ab3d 0x55a01f41c458 0x55a01f41702f 0x55a01f3a9aba 0x55a01f41c2c0 0x55a01f41702f 0x55a01f3a9aba 0x55a01f418cd4 0x55a01f49b986 0x55a01f418350 0x55a01f49b986 0x55a01f418350 0x55a01f49b986 0x55a01f418350 0x55a01f3a9f19 0x55a01f3eda79 0x55a01f3a8b32 0x55a01f41c1dd 0x55a01f41702f 0x55a01f3a9aba 0x55a01f418cd4 0x55a01f41702f 0x55a01f3a9aba 0x55a01f417eae 0x55a01f3a99da 0x55a01f418108 0x55a01f41702f
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:12:02tcmalloc: large alloc 1434370048 bytes == 0x55a09d2a6000 @  0x7f94aff8d615 0x55a01f3a53bc 0x55a01f48618a 0x55a01f3a81cd 0x55a01f49ab3d 0x55a01f41c458 0x55a01f41702f 0x55a01f3a9aba 0x55a01f41c2c0 0x55a01f41702f 0x55a01f3a9aba 0x55a01f418cd4 0x55a01f49b986 0x55a01f4183

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
import numpy as np
from margin_loss_pt import LargeMarginLoss

In [8]:
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

1.9.0+cu111


device(type='cuda', index=0)

In [9]:
def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, *_ = model(data)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            _, idx = output.max(dim=1)
            correct += (idx == target).sum().item()

    print('Test set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
from torch.utils import data
from torchvision import datasets
from torchvision import transforms

train_loader = data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=256, shuffle=True, drop_last=True)

test_loader = data.DataLoader(
        datasets.MNIST('./data', train=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=2048, shuffle=False, drop_last=False)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(True)
        )
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        flatten = conv2.view(x.shape[0], -1)        
        fc1 = self.fc1(flatten)
        fc2 = self.fc2(fc1)
        return fc2, [conv1, conv2]

In [13]:
lm = LargeMarginLoss(
    gamma=10000,
    alpha_factor=4,
    top_k=1,
    dist_norm=np.inf
)

def train_lm(model, train_loader, optimizer, epoch, lm):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        one_hot = torch.zeros(len(target), 10).scatter_(1, target.unsqueeze(1), 1.).float()
        one_hot = one_hot.cuda()
        optimizer.zero_grad()
        output, feature_maps = model(data)
        #loss = F.mse_loss(output, target) * 5e-4 # l2_loss_weght
        loss = lm(output, one_hot, feature_maps)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# net = Net().to(device)
net = nn.DataParallel(net).to(device)
optim = Adam(net.parameters())
for i in range(0, 5):
    train_lm(net, train_loader, optim, i, lm)
    test(net, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: -3864.983398
Train Epoch: 0 [25600/60000 (43%)]	Loss: -4168.206055
Train Epoch: 0 [51200/60000 (85%)]	Loss: -4269.446289
Test set: Accuracy: 8887/10000 (89%)

Train Epoch: 1 [0/60000 (0%)]	Loss: -4106.529297
Train Epoch: 1 [25600/60000 (43%)]	Loss: -4173.661133
Train Epoch: 1 [51200/60000 (85%)]	Loss: -4211.122070
Test set: Accuracy: 8896/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -4267.726074
Train Epoch: 2 [25600/60000 (43%)]	Loss: -4721.286133
Train Epoch: 2 [51200/60000 (85%)]	Loss: -4668.863281
Test set: Accuracy: 9898/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: -4893.762207
Train Epoch: 3 [25600/60000 (43%)]	Loss: -4816.452148
Train Epoch: 3 [51200/60000 (85%)]	Loss: -4853.994141
Test set: Accuracy: 9904/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: -4753.899414
Train Epoch: 4 [25600/60000 (43%)]	Loss: -4865.370117
Train Epoch: 4 [51200/60000 (85%)]	Loss: -4841.532715
Test set: Accuracy: 9909/10000 (99%)



In [ ]:
def train_ce(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, _ = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

net = Net().to(device)
# net = nn.DataParallel(net).to(device)
optim = Adam(net.parameters())
for i in range(0, 5):    
    train_ce(net, train_loader, optim, i)
    test(net, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.312209
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.057380
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.069225
Test set: Accuracy: 9871/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.021330
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.058321
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.032983
Test set: Accuracy: 9887/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.020935
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.051123
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.020498
Test set: Accuracy: 9911/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.018395
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.030107
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.018094
Test set: Accuracy: 9903/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.016279
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.013521
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.013503
Test set: Accuracy: 9915/10000 (99%)

